# Simple iteration for systems of linear equations

First, generate a random diagonally dominant matrix, for testing.

In [1]:
import numpy as np
rndm = np.random.RandomState(1234)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

# I.  Jacobi iteration

Given

$$
A x = b
$$

separate the diagonal part $D$,

$$ A = D + (A - D) $$

and write

$$
x = D^{-1} (D - A) x + D^{-1} b\;.
$$

Then iterate

$$
x_{n + 1} = B x_{n} + c\;,
$$

where 

$$
B = D^{-1} (A - D) \qquad \text{and} \qquad c = D^{-1} b
$$


Let's construct the matrix and the r.h.s. for the Jacobi iteration

In [2]:
diag_1d = np.diag(A)

B = -A.copy()
np.fill_diagonal(B, 0)

D = np.diag(diag_1d)
invD = np.diag(1./diag_1d)
BB = invD @ B 
c = invD @ b

In [3]:
# sanity checks
from numpy.testing import assert_allclose

assert_allclose(-B + D, A)


# xx is a "ground truth" solution, compute it using a direct method
xx = np.linalg.solve(A, b)

np.testing.assert_allclose(A@xx, b)
np.testing.assert_allclose(D@xx, B@xx + b)
np.testing.assert_allclose(xx, BB@xx + c)

Check that $\| B\| \leqslant 1$:

In [4]:
np.linalg.norm(BB)


0.36436161983015336

### Do the Jacobi iteration

In [5]:
n_iter = 50

x0 = np.ones(n)
x = x0
for _ in range(n_iter):
    x = BB @ x + c

In [6]:
# Check the result:

A @ x - b

array([ 1.11022302e-16,  0.00000000e+00, -2.22044605e-16, -1.11022302e-16,
        1.11022302e-16,  0.00000000e+00, -2.08166817e-17,  0.00000000e+00,
       -2.77555756e-17,  1.11022302e-16])

### Task I.1

Collect the proof-of-concept above into a single function implementing the Jacobi iteration. This function should receive the r.h.s. matrix $A$, the l.h.s. vector `b`, and the number of iterations to perform.


The matrix $A$ in the illustration above is strongly diagonally dominant, by construction. 
What happens if the diagonal matrix elements of $A$ are made smaller? Check the convergence of the Jacobi iteration, and check the value of the norm of $B$.

(20% of the total grade)


In [7]:
from sklearn.metrics import mean_absolute_error


def Jacobi_iterate(A: np.array, b: np.array, eps=1e-15, n_max_iters=1000):
    """
        Arguments:
            A (np.array): initial A matrix
            b (np.array): initial array
            eps (float): minimal error value
            n_max_iter (int): maximum amount of iterations need to be done
            
        Outputs:
            n_iter (int): amount of iterations done
            x (np.array): vector
    """
    # 1. Matrix splitting
    D, U, L = np.diag(np.diag(A)), np.triu(A, 1), np.tril(A, -1)

    # 2. Building an inverse matrix
    D_inv = np.linalg.inv(D)

    # 3. Defining objective function
    objective = lambda A, x, b: mean_absolute_error((A @ x), b)
    
    # 4. Initialization of starting x
    x_prev = np.zeros(shape=(A.shape[0]))
    x = np.random.uniform(size=(A.shape[0]))

    # 5. Computing optimization by precomputed components
    x_multiplier = D_inv @ (-L - U)
    offset = D_inv @ b
    
    # 6. Start iteration = 0
    n_iter = 0
    while objective(A, x, b) > eps and n_iter < n_max_iters:
        x_prev = x
        x = x_multiplier @ x + offset
        n_iter += 1
        print(f"[{n_iter}]: {objective(A, x, b)}")
        
    return n_iter, x

_, x = Jacobi_iterate(A, b, n_max_iters = 2000)

[1]: 2.8410743731186243
[2]: 0.8479323565755781
[3]: 0.2547348695344968
[4]: 0.07634618890098839
[5]: 0.022900676851471114
[6]: 0.006867754336327791
[7]: 0.002059687178940594
[8]: 0.0006177097378785047
[9]: 0.00018525415925357627
[10]: 5.555862712606427e-05
[11]: 1.6662302901770954e-05
[12]: 4.997105861920259e-06
[13]: 1.4986564034119575e-06
[14]: 4.4945435990796646e-07
[15]: 1.3479355313525399e-07
[16]: 4.042524356720478e-08
[17]: 1.2123727582968868e-08
[18]: 3.6359650134459853e-09
[19]: 1.0904436263670104e-09
[20]: 3.270293816998171e-10
[21]: 9.807765745384334e-11
[22]: 2.941400595313226e-11
[23]: 8.821348512766392e-12
[24]: 2.6456288548803996e-12
[25]: 7.933778634061639e-13
[26]: 2.3799746895480924e-13
[27]: 7.136617685699065e-14
[28]: 2.1447427167586853e-14
[29]: 6.4254157550180935e-15
[30]: 1.9088897129648787e-15
[31]: 5.651729084732438e-16


In [8]:
# Verficiation of convergence to actual x
np.allclose(A @ x, b)

True

In [9]:
## NOW lets see, what happens if we reduce our diagonal
_, x = Jacobi_iterate((A - np.diag([10] * n)), b, n_max_iters = 2000)

[1]: 2.7390200086803658
[2]: 2.3321732866037275
[3]: 1.9801574181986126
[4]: 1.684155006659644
[5]: 1.4313785804054158
[6]: 1.2167853760103813
[7]: 1.0343224359942549
[8]: 0.8792253782439456
[9]: 0.7473850729589346
[10]: 0.6353142052865632
[11]: 0.5400484557229012
[12]: 0.45906786125583005
[13]: 0.39023035592934646
[14]: 0.33171507655148746
[15]: 0.2819741989186725
[16]: 0.23969199615978037
[17]: 0.2037500354423515
[18]: 0.17319759361087605
[19]: 0.14722650902850595
[20]: 0.12514980438711293
[21]: 0.10638351504415533
[22]: 0.09043124221067869
[23]: 0.07687102239827445
[24]: 0.06534416580045851
[25]: 0.05554576836555332
[26]: 0.04721664659001657
[27]: 0.040136481694420065
[28]: 0.03411799183441235
[29]: 0.029001978192195528
[30]: 0.024653113909600465
[31]: 0.020956364473209886
[32]: 0.017813944864911863
[33]: 0.01514273298957916
[34]: 0.01287207096084288
[35]: 0.01094189608540273
[36]: 0.009301152107377097
[37]: 0.007906438687530364
[38]: 0.0067208633939118525
[39]: 0.005713065837197352

<b>Summary</b>: Jacobi iteration applicable only to diagonal dominant matrix, otherwise algorithm will not converge.

# II. Seidel's iteration.

##### Task II.1

Implement the Seidel's iteration. 

Test it on a random matrix. Study the convergence of iterations, relate to the norm of the iteration matrix.

(30% of the total grade)

In [10]:
def Gauss_Seidel_iterate(A: np.array, b: np.array, eps=1e-15, n_max_iters=1000):
    """
        Arguments:
            A (np.array): initial A matrix
            b (np.array): initial array
            eps (float): minimal error value
            n_max_iter (int): maximum amount of iterations need to be done
            
        Outputs:
            n_iter (int): amount of iterations done
            x (np.array): vector
    """
    # 1. Matrix splitting
    U, LD = np.triu(A, 1), np.tril(A)
    LD_inv = np.linalg.inv(LD)
    print(f"Norm: {np.linalg.norm(LD_inv)}")

    # 2. Defining objective function
    objective = lambda A, x, b: mean_absolute_error((A @ x), b)

    # 3. Initialization of starting x
    x = np.random.uniform(size=(A.shape[0]))

    # 4. definition of converging function
    converge = lambda LD_inv, x, U, b: LD_inv @ (-U @ x + b)

    # 5. Start iteration = 0
    n_iter = 0
    while objective(A, x, b) > eps and n_iter < n_max_iters:
        x = converge(LD_inv, x, U, b)
        n_iter += 1
        print(f"[{n_iter}]: {objective(A, x, b)}")
        
    return n_iter, x

_, x = Gauss_Seidel_iterate(A, b, n_max_iters = 2000)
np.allclose(A @ x, b)

Norm: 0.20499614567534405
[1]: 1.0795137671697208
[2]: 0.05694484773511638
[3]: 0.0023699145091462363
[4]: 0.00020845740229980537
[5]: 1.0319481962708239e-05
[6]: 6.705665368311742e-07
[7]: 4.551915503969717e-08
[8]: 2.084844997335411e-09
[9]: 1.8106093846448345e-10
[10]: 8.442512314243977e-12
[11]: 6.236546001847643e-13
[12]: 3.5836958400814465e-14
[13]: 1.8301332671555315e-15
[14]: 2.0816681711721685e-16


True

In [11]:
## Lets research how reduction of diagonal dominance influence on convergence speed
_, x = Gauss_Seidel_iterate((A - np.diag([10] * n)), b, n_max_iters = 2000)

Norm: 0.5898638843769459
[1]: 1.2508289388415395
[2]: 0.16463081532453047
[3]: 0.03822346634549188
[4]: 0.006309988081376339
[5]: 0.0014513317224728486
[6]: 0.0002566099683093487
[7]: 5.6688196258027763e-05
[8]: 1.0066583914559654e-05
[9]: 2.296167380658881e-06
[10]: 4.0577968697594337e-07
[11]: 8.979565289460311e-08
[12]: 1.6712449203357128e-08
[13]: 3.393946922114477e-09
[14]: 6.702322068868893e-10
[15]: 1.3005858646997214e-10
[16]: 2.8488921638425423e-11
[17]: 5.173511272160703e-12
[18]: 1.1722536136238303e-12
[19]: 2.0132298605979316e-13
[20]: 4.654887586497125e-14
[21]: 7.881889585448221e-15
[22]: 1.802030746844707e-15
[23]: 3.368832990346959e-16


# III. Minimum residual scheme

### Task III.1

Implement the $\textit{minimum residual}$ scheme: an explicit non-stationary method, where at each step you select the iteration parameter $\tau_n$ to minimize the residual $\mathbf{r}_{n+1}$ given $\mathbf{r}_n$. Test it on a random matrix, study the convergence to the solution, in terms of the norm of the residual and the deviation from the ground truth solution (which you can obtain using a direct method). Study how the iteration parameter $\tau_n$ changes as iterations progress.

(50% of the grade)

In [93]:
def minimum_residual_convergence(A: np.array, b: np.array, eps=1e-15, n_max_iters=1000):
    """
        Arguments:
            A (np.array): initial A matrix
            b (np.array): initial array
            eps (float): minimal error value
            n_max_iter (int): maximum amount of iterations need to be done
            
        Outputs:
            n_iter (int): amount of iterations done
            x (np.array): vector
    """

    # 1. Functions definition & initial computings
    f_residuals = lambda A, x, b: A @ x - b
    f_tao = lambda A, r: np.dot(r, A * r)/np.sum(np.power((A @ r),2))
    objective = lambda residuals: np.linalg.norm(residuals)
    
    # 2. Initialization of starting x
    x = np.random.uniform(size=(A.shape[0]))         
    r = f_residuals(A, x, b)

    # 3. Start iteration = 0
    n_iter = 0
    while objective(r) > eps and n_iter < n_max_iters:

        # 1. Offsetting components
        r = f_residuals(A, x, b)
        
        tao = f_tao(A, r)

        # 2. Computing
        x = x - r * tao
        n_iter += 1
        print(f"[{n_iter}]: {objective(r)}")

    return n_iter, x
    

In [94]:
_, x = minimum_residual_convergence(A, b)

[1]: 34.33492748166756
[2]: 29.775401199375608
[3]: 25.925193237331435
[4]: 22.6380138839721
[5]: 19.81172488567009
[6]: 17.369450080648942
[7]: 15.250746384515368
[8]: 13.40685541344096
[9]: 11.79782602045473
[10]: 10.390585519229262
[11]: 9.157541189744393
[12]: 8.075511751219892
[13]: 7.124888036914044
[14]: 6.288967460984515
[15]: 5.553426687473964
[16]: 4.905905426370702
[17]: 4.335678619440467
[18]: 3.8333975955232353
[19]: 3.3908840690122584
[20]: 3.0009641483195066
[21]: 2.6573325573069777
[22]: 2.3544398397155533
[23]: 2.0873973326630946
[24]: 1.8518961864297936
[25]: 1.6441377578530367
[26]: 1.46077341398883
[27]: 1.2988522447047353
[28]: 1.155775475618329
[29]: 1.0292565586339653
[30]: 0.917286045159359
[31]: 0.8181004538464463
[32]: 0.7301544550466482
[33]: 0.6520958192147607
[34]: 0.5827427137332081
[35]: 0.5210630687791195
[36]: 0.46615584985363684
[37]: 0.4172341569250691
[38]: 0.37361011066307864
[39]: 0.33468148819664617
[40]: 0.29992004567775127
[41]: 0.26886142764600

In [95]:
np.allclose(A @ x, b)

True

In [96]:
## Lets research how reduction of diagonal dominance influence on convergence speed
_, x = minimum_residual_convergence((A - np.diag([15] * n)), b, n_max_iters = 2000)

[1]: 4.149948264519194
[2]: 3.702067595935181
[3]: 3.301103814241516
[4]: 2.9421998961814175
[5]: 2.6210296330043223
[6]: 2.3337591223305894
[7]: 2.0770277739646885
[8]: 1.847966575807005
[9]: 1.6443063553983042
[10]: 1.4647657746191707
[11]: 1.3106482410998506
[12]: 1.1963403731500684
[13]: 0.8600408155494154
[14]: 0.8304835867109049
[15]: 0.7617059990692348
[16]: 0.716129283502198
[17]: 0.6591863437041944
[18]: 0.6244770878506348
[19]: 0.6009244294335846
[20]: 0.5878844513034314
[21]: 0.6061284837063683
[22]: 0.5687609179521729
[23]: 0.5558482925616831
[24]: 0.562411714336138
[25]: 0.5346343847136874
[26]: 0.5224790527861528
[27]: 0.5200549790243623
[28]: 0.5033242187155009
[29]: 0.49421205056781964
[30]: 0.4864523562453602
[31]: 0.4880962568011177
[32]: 0.4714720639202094
[33]: 0.46354365239895456
[34]: 0.4568119563832179
[35]: 0.45080320620339676
[36]: 0.4457494903701797
[37]: 0.4416184376821286
[38]: 0.4372088279005107
[39]: 0.4362836583014104
[40]: 0.42880102835114703
[41]: 0.425

[900]: 0.07749597667919696
[901]: 0.0739585446733711
[902]: 0.07405067906361336
[903]: 0.08324817821867647
[904]: 0.07727842432108543
[905]: 0.073742392349033
[906]: 0.073824394455914
[907]: 0.08301935483194028
[908]: 0.07705919595394425
[909]: 0.07352377845965465
[910]: 0.07359557573553602
[911]: 0.08278746290353041
[912]: 0.07683671523350709
[913]: 0.07330234988707639
[914]: 0.07336424268517529
[915]: 0.08255110223637291
[916]: 0.07661010808604801
[917]: 0.07307790500735471
[918]: 0.07313046270650997
[919]: 0.08230952090224813
[920]: 0.07637893729585414
[921]: 0.07285038759822011
[922]: 0.0728943750563505
[923]: 0.08206235187991476
[924]: 0.07614303433693706
[925]: 0.07261985378111689
[926]: 0.0726561819169045
[927]: 0.08180948950558109
[928]: 0.07590241610295663
[929]: 0.07238644560297296
[930]: 0.07241613080765495
[931]: 0.08155100958726427
[932]: 0.07565722757049502
[933]: 0.072150366081679
[934]: 0.07217449633498403
[935]: 0.08128711742704008
[936]: 0.07540770266034251
[937]: 0.0

[1399]: 0.06347789228812124
[1400]: 0.06037491908357675
[1401]: 0.060610865344903626
[1402]: 0.06344834197891307
[1403]: 0.06035010928506914
[1404]: 0.060582968035294574
[1405]: 0.06341797696385666
[1406]: 0.06032449875681996
[1407]: 0.06055439220479485
[1408]: 0.0633868114048076
[1409]: 0.060298099656511735
[1410]: 0.060525148425086744
[1411]: 0.06335486070337314
[1412]: 0.06027092531114866
[1413]: 0.0604952481882986
[1414]: 0.06332214134265284
[1415]: 0.06024299008456437
[1416]: 0.06046470380332444
[1417]: 0.06328867073676592
[1418]: 0.060214309251383984
[1419]: 0.06043352829773148
[1420]: 0.06325446708880779
[1421]: 0.06018489887796136
[1422]: 0.06040173532552883
[1423]: 0.06321954925768851
[1424]: 0.06015477571064306
[1425]: 0.06036933908095141
[1426]: 0.06318393663412007
[1427]: 0.06012395707155561
[1428]: 0.060336354218312536
[1429]: 0.06314764902586747
[1430]: 0.0600924607619944
[1431]: 0.06030279577789669
[1432]: 0.06311070655223876
[1433]: 0.0600603049733813
[1434]: 0.06026867

<b>Summary</b>: From the illustrations above seen the similar behaviour with Jacobi method
